In [1]:
from pathlib import Path
import polars as pl

from record_consolidation.df_consolidations import (
    normalize_subset,
    # normalize_subset_via_joins,
    normalize_subset_via_joins,
)

In [5]:
root_path = Path("test_data/normalize_subset")
input_path = root_path / "input.parquet"
snapshot_path = root_path / "processed.parquet"

raw_input: pl.DataFrame = pl.read_parquet(input_path)
COLS_TO_NORMALIZE = ["issuer_name", "cusip", "isin", "figi"]
locally_processed: pl.DataFrame = normalize_subset_via_joins(
    raw_input,
    cols_to_normalize=COLS_TO_NORMALIZE,
)
snapshot: pl.DataFrame = pl.read_parquet(snapshot_path)

hconcated = pl.concat(
    [locally_processed.rename(lambda x: f"{x}_normed"), raw_input],
    how="horizontal",
)
for col in COLS_TO_NORMALIZE:
    print(f"{col}=")
    non_nulls = hconcated.filter(pl.col(col).is_not_null())
    non_nulls_unequal = hconcated.filter(
        pl.col(col).is_not_null() & (pl.col(col) != pl.col(f"{col}_normed"))
    )
    display(non_nulls_unequal.head())
    assert non_nulls_unequal.height == 0, non_nulls_unequal

issuer_name=


issuer_name_normed,cusip_normed,isin_normed,figi_normed,rand1_normed,rand2_normed,issuer_name,cusip,isin,figi,rand1,rand2
str,str,str,str,i32,u32,str,str,str,str,i32,u32


cusip=


issuer_name_normed,cusip_normed,isin_normed,figi_normed,rand1_normed,rand2_normed,issuer_name,cusip,isin,figi,rand1,rand2
str,str,str,str,i32,u32,str,str,str,str,i32,u32
"""MICROSOFT CORPORATION""","""594918104""","""US5949181045""","""MSFT""",10,51,"""MICROSOFT CORPORATION""","""594918105""","""US5949181045""",null,10,51
"""AMAZON.COM, INC.""","""023135106""","""US0231351067""","""AMZN""",10,57,"""AMAZON.COM, INC.""","""23135106""","""US0231351067""",null,10,57
"""AMAZON.COM, INC.""","""023135106""","""US0231351067""","""AMZN""",10,146,"""AMAZON.COM, INC.""","""23135106""","""US0231351067""",null,10,146
"""AMAZON.COM, INC.""","""023135106""","""US0231351067""","""AMZN""",10,280,"""AMAZON.COM, INC.""","""23135106""","""US0231351067""",null,10,280
"""AMAZON.COM, INC.""","""023135106""","""US0231351067""","""AMZN""",10,412,"""AMAZON.COM, INC.""","""23135106""","""US0231351067""",null,10,412


AssertionError: shape: (5, 12)
┌──────────────┬──────────────┬─────────────┬─────────────┬───┬─────────────┬──────┬───────┬───────┐
│ issuer_name_ ┆ cusip_normed ┆ isin_normed ┆ figi_normed ┆ … ┆ isin        ┆ figi ┆ rand1 ┆ rand2 │
│ normed       ┆ ---          ┆ ---         ┆ ---         ┆   ┆ ---         ┆ ---  ┆ ---   ┆ ---   │
│ ---          ┆ str          ┆ str         ┆ str         ┆   ┆ str         ┆ str  ┆ i32   ┆ u32   │
│ str          ┆              ┆             ┆             ┆   ┆             ┆      ┆       ┆       │
╞══════════════╪══════════════╪═════════════╪═════════════╪═══╪═════════════╪══════╪═══════╪═══════╡
│ MICROSOFT    ┆ 594918104    ┆ US594918104 ┆ MSFT        ┆ … ┆ US594918104 ┆ null ┆ 10    ┆ 51    │
│ CORPORATION  ┆              ┆ 5           ┆             ┆   ┆ 5           ┆      ┆       ┆       │
│ AMAZON.COM,  ┆ 023135106    ┆ US023135106 ┆ AMZN        ┆ … ┆ US023135106 ┆ null ┆ 10    ┆ 57    │
│ INC.         ┆              ┆ 7           ┆             ┆ 

In [2]:
mp = Path("/Users/mgaster/repos/sec-parsing-deployment/trash/meetings.parquet")

In [3]:
meetings = pl.read_parquet(mp)

In [8]:
normed_meetings = normalize_subset_via_joins(
    meetings, ["issuer_name", "cusip", "isin", "figi"]
)

In [13]:
meetings

issuer_name,cusip,isin,figi,meeting_date
str,str,str,str,str
"""DECKERS OUTDOOR CORPORATION""","""243537107""","""US2435371073""",null,"""09/11/2023"""
"""CSX CORPORATION""","""126408103""","""US1264081035""",null,"""05/08/2024"""
"""CANADIAN PACIFIC KANSAS CITY L…","""13646K108""","""CA13646K1084""",null,"""04/24/2024"""
"""VISA INC.""","""92826C839""","""US92826C8394""",null,"""01/23/2024"""
"""AMAZON.COM, INC.""","""023135106""","""US0231351067""",null,"""05/22/2024"""
…,…,…,…,…
"""Nutraceutical InvestCo""",null,null,null,"""N/A"""
"""Foundation Capital V LP""",null,null,null,"""N/A"""
"""Uno Co-Invest LP""",null,null,null,"""N/A"""


In [11]:
meetings.select(pl.all().is_null().sum())

issuer_name,cusip,isin,figi,meeting_date
u32,u32,u32,u32,u32
0,5780,73878,242631,0


In [12]:
normed_meetings.select(pl.all().is_null().sum())

issuer_name,cusip,isin,figi,meeting_date
u32,u32,u32,u32,u32
0,855,566,154342,0


In [4]:
meetings.filter(pl.col("figi").is_not_null())

issuer_name,cusip,isin,figi,meeting_date
str,str,str,str,str
"""Nvidia Corporation""","""67066G104""","""US67066G1040""","""BBG000BBJQV0""","""06/26/2024"""
"""Applied Materials, Inc.""","""038222105""","""US0382221051""","""BBG000BBPFB9""","""03/07/2024"""
"""Apple, Inc.""","""037833100""","""US0378331005""","""BBG000B9XRY4""","""02/28/2024"""
"""KINDER MORGAN INC""","""49456B101""","""US49456B1017""","""BBG0019JZ882""","""05/08/2024"""
"""INTEL CORPORATION CORP""","""458140100""","""US4581401001""","""BBG000C0G1D1""","""05/07/2024"""
…,…,…,…,…
"""Brookfield Asset Management Lt…","""113004105""","""US1130041056""","""BBG01BPHNXZ3""","""06/07/2024"""
"""Applied Materials Inc""","""038222105""","""US0382221051""","""BBG000BBPFB9""","""03/07/2024"""
"""The Home Depot Inc""","""437076102""","""US4370761029""","""BBG000BKZB36""","""05/16/2024"""


In [5]:
from record_consolidation.df_consolidations import extract_normalized_atomic


extract_normalized_atomic(meetings.head(1000))

isin,issuer_name,meeting_date,cusip
str,str,str,str
"""US2435371073""","""DECKERS OUTDOOR CORPORATION""","""09/11/2023""","""243537107"""
"""CNE1000004S4""","""Zhejiang Expressway Co.""","""05/08/2024""","""126408103"""
"""USN070592100""","""ASML HOLDINGS N.V.""","""04/24/2024""","""N07059210"""
"""US92826C8394""","""VISA INC.""","""01/23/2024""","""92826C839"""
"""US0231351067""","""AMAZON.COM, INC.""","""05/22/2024""","""023135106"""
…,…,…,…
null,"""NetApp, Inc.""","""09/13/2023""","""64110D104"""
"""INE220B01022""","""Kalpataru Projects Internation…","""07/17/2023""",null
null,"""APA Corporation""","""03/27/2024""","""03743Q108"""


In [6]:
meetings.head(1000)

issuer_name,cusip,isin,figi,meeting_date
str,str,str,str,str
"""DECKERS OUTDOOR CORPORATION""","""243537107""","""US2435371073""",null,"""09/11/2023"""
"""CSX CORPORATION""","""126408103""","""US1264081035""",null,"""05/08/2024"""
"""CANADIAN PACIFIC KANSAS CITY L…","""13646K108""","""CA13646K1084""",null,"""04/24/2024"""
"""VISA INC.""","""92826C839""","""US92826C8394""",null,"""01/23/2024"""
"""AMAZON.COM, INC.""","""023135106""","""US0231351067""",null,"""05/22/2024"""
…,…,…,…,…
"""ONO PHARMACEUTICAL CO.,LTD.""",null,"""JP3197600004""",null,"""06/20/2024"""
"""PennantPark Companies""","""708062104""",null,null,"""02/06/2024"""
"""China Reinsurance (Group) Corp…",null,"""CNE100002342""",null,"""03/04/2024"""


In [7]:
normalize_subset_via_joins(
    meetings.head(1000),
    ["issuer_name", "cusip", "isin", "figi"],
)

col='figi' wasn't used.


issuer_name,cusip,isin,figi,meeting_date
str,str,str,str,str
"""DECKERS OUTDOOR CORPORATION""","""243537107""","""US2435371073""",null,"""09/11/2023"""
"""CSX CORPORATION""","""126408103""","""US1264081035""",null,"""05/08/2024"""
"""CANADIAN PACIFIC KANSAS CITY L…","""13646K108""","""CA13646K1084""",null,"""04/24/2024"""
"""VISA INC.""","""92826C839""","""US92826C8394""",null,"""01/23/2024"""
"""AMAZON.COM, INC.""","""023135106""","""US0231351067""",null,"""05/22/2024"""
…,…,…,…,…
"""ONO PHARMACEUTICAL CO.,LTD.""",null,"""JP3197600004""",null,"""06/20/2024"""
"""PennantPark Companies""","""708062104""",null,null,"""02/06/2024"""
"""China Reinsurance (Group) Corp…",null,"""CNE100002342""",null,"""03/04/2024"""


In [6]:
normed2 = normalize_subset_via_joins(
    meetings.filter(pl.col("figi").is_not_null()),
    ["issuer_name", "cusip", "isin", "figi"],
)

In [7]:
meetings.filter(pl.col("figi").is_not_null())

issuer_name,cusip,isin,figi,meeting_date
str,str,str,str,str
"""Nvidia Corporation""","""67066G104""","""US67066G1040""","""BBG000BBJQV0""","""06/26/2024"""
"""Applied Materials, Inc.""","""038222105""","""US0382221051""","""BBG000BBPFB9""","""03/07/2024"""
"""Apple, Inc.""","""037833100""","""US0378331005""","""BBG000B9XRY4""","""02/28/2024"""
"""KINDER MORGAN INC""","""49456B101""","""US49456B1017""","""BBG0019JZ882""","""05/08/2024"""
"""INTEL CORPORATION CORP""","""458140100""","""US4581401001""","""BBG000C0G1D1""","""05/07/2024"""
…,…,…,…,…
"""Brookfield Asset Management Lt…","""113004105""","""US1130041056""","""BBG01BPHNXZ3""","""06/07/2024"""
"""Applied Materials Inc""","""038222105""","""US0382221051""","""BBG000BBPFB9""","""03/07/2024"""
"""The Home Depot Inc""","""437076102""","""US4370761029""","""BBG000BKZB36""","""05/16/2024"""


In [8]:
normed2  # .filter(pl.col("canonical_row").is_null())

issuer_name,cusip,isin,figi,meeting_date
str,str,str,str,str
"""Nvidia Corporation""","""67066G104""","""US67066G1040""","""BBG000BBJQV0""","""06/26/2024"""
"""Applied Materials, Inc.""","""038222105""","""US0382221051""","""BBG000BBPFB9""","""03/07/2024"""
"""Apple, Inc.""","""037833100""","""US0378331005""","""BBG000B9XRY4""","""02/28/2024"""
"""KINDER MORGAN INC""","""49456B101""","""US49456B1017""","""BBG0019JZ882""","""05/08/2024"""
"""Intel Corporation""","""458140100""","""US4581401001""","""BBG000C0G1D1""","""05/07/2024"""
…,…,…,…,…
"""Brookfield Asset Management Lt…","""113004105""","""US1130041056""","""BBG01BPHNXZ3""","""06/07/2024"""
"""Applied Materials, Inc.""","""038222105""","""US0382221051""","""BBG000BBPFB9""","""03/07/2024"""
"""HOME DEPOT INC""","""437076102""","""US4370761029""","""BBG000BKZB36""","""05/16/2024"""


In [11]:
normed2.select(pl.col("canonical_row")).to_series()[0]

{'issuer_name': 'Coca-Cola Co',
 'figi': 'BBG000BMX289',
 'isin': 'US1912161007',
 'cusip': '191216100'}

In [5]:
normed = normalize_subset(meetings, ["issuer_name", "cusip", "isin", "figi"])

col='figi' wasn't used.


In [7]:
normed.select(pl.struct(pl.all()))

issuer_name
struct[5]
"{""DECKERS OUTDOOR CORPORATION"",""243537107"",""US2435371073"",null,""09/11/2023""}"
"{""DECKERS OUTDOOR CORPORATION"",""243537107"",""US2435371073"",null,""05/08/2024""}"
"{""DECKERS OUTDOOR CORPORATION"",""243537107"",""US2435371073"",null,""04/24/2024""}"
"{""DECKERS OUTDOOR CORPORATION"",""243537107"",""US2435371073"",null,""01/23/2024""}"
"{""DECKERS OUTDOOR CORPORATION"",""243537107"",""US2435371073"",null,""05/22/2024""}"
…
"{""DECKERS OUTDOOR CORPORATION"",""243537107"",""US2435371073"",null,""N/A""}"
"{""DECKERS OUTDOOR CORPORATION"",""243537107"",""US2435371073"",null,""N/A""}"
"{""DECKERS OUTDOOR CORPORATION"",""243537107"",""US2435371073"",null,""N/A""}"


In [15]:
dict(
    pl.DataFrame(
        {
            "a": [
                1,
                2,
            ]
        },
    ).schema
)

{'a': Int64}